# Intro
WeRateDogs is a Twitter account that posts and rates pictures of dogs. These ratings often are not serious and have numerators that are greater than the denominators. 

In the analysis, I will aim to (1) wrangle the data + (2) answer the following questions:

1. What is WeRateDogs's posting trend by month?
2. What is the monthly trend of interactions with WeRateDogs's posts?
3. What are the most popular dog breeds based on number of posts, interactions by Twitter users, and ratings?
4. Is there any correlation between WeRateDogs's ratings and the interactions by Twitter users?

In [691]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os 
import json 
import re
pd.set_option('display.max_colwidth', None)

# 1. Gathering Data

In [692]:
#1.  Archive file
archive = pd.read_csv('/Users/timtism/Documents/Github Repo/Data Analytics Portfolio/Python/Projects/Twitter WeRateDogs Data Set/datasets/twitter-archive-enhanced.csv')
#2. Image prediction file (URL)
    # Predictions of dog breeds
url_image_predictions = ('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/'
                         '599fd2ad_image-predictions/image-predictions.tsv')

    # Create dataframe from TSV
predictions = pd.read_csv(url_image_predictions, delimiter='\t')

#3. Tweets
tweet = pd.read_json('/Users/timtism/Documents/Github Repo/Data Analytics Portfolio/Python/Projects/Twitter WeRateDogs Data Set/datasets/tweet-json.txt',  lines = True)

---

# 2. Assessing  Data

### a. `archive`

In [693]:
archive.info()
archive.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        1611 non-null   object 
 13  doggo                       97 no

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1664,682788441537560576,NaN,NaN,2016-01-01 05:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Happy New Year from your fav holiday squad! 🎉 12/10 for all\n\nHere's to a pupper-filled year 🍻🐶🐶🐶 https://t.co/ZSdEj59FGf,NaN,NaN,NaN,https://twitter.com/dog_rates/status/682788441537560576/photo/1,12,10,NaN,NaN,NaN,pupper,NaN
1002,747885874273214464,NaN,NaN,2016-06-28 20:14:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a mighty rare blue-tailed hammer sherk. Human almost lost a limb trying to take these. Be careful guys. 8/10 https://t.co/TGenMeXreW,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/747885874273214464/photo/1,https://twitter.com/dog_rates/status/747885874273214464/photo/1",8,10,a,NaN,NaN,NaN,NaN
630,794332329137291264,NaN,NaN,2016-11-04 00:15:59 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Nimbus (like the cloud). He just bought this fancy af duck raincoat. Only protects one ear tho. 12/10 so h*ckin floofy https://t.co/SIQbb8c3AU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/794332329137291264/photo/1,12,10,Nimbus,NaN,NaN,NaN,NaN
175,857989990357356544,NaN,NaN,2017-04-28 16:08:49 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Rosie. She was just informed of the walk that's about to happen. Knows there are many a stick along the way. 12/10 such excite https://t.co/sOl7cFaP5X,NaN,NaN,NaN,https://twitter.com/dog_rates/status/857989990357356544/photo/1,12,10,Rosie,NaN,NaN,NaN,NaN
1213,715220193576927233,NaN,NaN,2016-03-30 16:52:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Nico. His selfie game is strong af. Excellent use of a sneaky tongue slip. 10/10 star material https://t.co/1OBdJkMOFx,NaN,NaN,NaN,https://twitter.com/dog_rates/status/715220193576927233/photo/1,10,10,Nico,NaN,NaN,NaN,NaN


### `archive` columns:
-  `tweet_id`: unique identifier for each tweet
- `in_reply_to_status_id`: original tweet_id if the row is a **reply**. If not, null
- `in_reply_to_user_id`: user id of the original tweet if the row  is a **reply**.  If not, null
- `timestamp`: time when this tweet was created
- `source`: HTML-formatted string of platform to post the tweet
-  `text`: content of the tweet
- `retweeted_status_id`: original tweet_id if the row is a **retweet**. If not, null
- `retweeted_status_user_id`: user id of the original tweet if the row is a **retweet**.  If not, null
- `expanded_urls`: tweet url
- `rating_numerator`: numerator of the rating of the dog. Note: ratings almost always greater than 10
- `rating_denominator`: denominator of the rating of the dog. Note: ratings always have a denominator of 10
- `name`: dog names
- `doggo` / `floofer` / `pupper` / `puppo`: one of the 4 dog stages

In [694]:
# From .info(), there are 181 retweets records (not null obs in retweeted_status_id)

# Check if there are any records in `archive` whose corresponding record with same tweet_id is missing in `predictions`
len(archive[~archive['tweet_id'].isin(predictions['tweet_id'])])

281

In [695]:
# Check the `rating_denominator` values:
archive['rating_denominator'].value_counts().sort_index()

rating_denominator
0         1
2         1
7         1
10     2333
11        3
15        1
16        1
20        2
40        1
50        3
70        1
80        2
90        1
110       1
120       1
130       1
150       1
170       1
Name: count, dtype: int64

In [696]:
# Check the `rating_numerator` values:
archive['rating_numerator'].value_counts().sort_index()

rating_numerator
0         2
1         9
2         9
3        19
4        17
5        37
6        32
7        55
8       102
9       158
10      461
11      464
12      558
13      351
14       54
15        2
17        1
20        1
24        1
26        1
27        1
44        1
45        1
50        1
60        1
75        2
80        1
84        1
88        1
99        1
121       1
143       1
144       1
165       1
182       1
204       1
420       2
666       1
960       1
1776      1
Name: count, dtype: int64

In [697]:
# Check dog name
archive['name'].value_counts().sort_index(ascending=False)

name
very            5
unacceptable    1
this            1
the             8
such            1
               ..
Aiden           1
Adele           1
Acro            1
Ace             1
Abby            2
Name: count, Length: 956, dtype: int64

In [698]:
archive['puppo'].value_counts()

puppo
puppo    30
Name: count, dtype: int64

In [699]:
# Create a column named `counter` that counts the number of dog stages each observation has

dog_stage = ['doggo','floofer','pupper','puppo']
archive['counter'] = 0

archive['counter'] += archive.apply(lambda row: sum(1 for col in dog_stage if not pd.isna(row[col])), axis = 1)

archive.head()



,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,counter
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,NaN,NaN,NaN,NaN,0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,NaN,NaN,NaN,NaN,0
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,NaN,NaN,NaN,NaN,0
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,NaN,NaN,NaN,NaN,0
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,NaN,NaN,NaN,NaN,0


In [700]:
# Check how many dogs have multiple dog stages
archive['counter'].value_counts() #-> 14 dogs

counter
0    1976
1     366
2      14
Name: count, dtype: int64

### b. `predictions`

In [701]:
predictions.info()
predictions.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
304,671518598289059840,https://pbs.twimg.com/media/CVG2l9jUYAAwg-w.jpg,1,Lakeland_terrier,0.428275,True,wire-haired_fox_terrier,0.111472,True,toy_poodle,0.105016,True
1557,793226087023144960,https://pbs.twimg.com/media/CwIa5CjW8AErZgL.jpg,1,wire-haired_fox_terrier,0.456047,True,Lakeland_terrier,0.273428,True,English_springer,0.083643,True
1665,812503143955202048,https://pbs.twimg.com/media/C0aXTLqXEAADxBi.jpg,2,loupe,0.546856,False,web_site,0.345298,False,bubble,0.010528,False
495,675740360753160193,https://pbs.twimg.com/ext_tw_video_thumb/675740268751138818/pu/img/dVaVeFAVT-lk_1ZV.jpg,1,golden_retriever,0.800495,True,kuvasz,0.097756,True,Saluki,0.068415,True
1099,720415127506415616,https://pbs.twimg.com/media/Cf9tuHUWsAAHSrV.jpg,1,Rottweiler,0.990312,True,black-and-tan_coonhound,0.002495,True,American_black_bear,0.001733,False


`predictions` columns:
- `tweet_id`: mentioned before
- `jpg_url`: dogs' image url
- `img_num`: the image number that corresponded to the most confident prediction (1 to 4, since tweets can have up to 4  images)
- `p1` / `p2` / `p3`: result of dogs that the #x (pX) algorithm to predict the image in the tweet
- `p1_conf` / `p2_conf` / `p3_conf`: how confident the algorithm is in its pX predictions
- `p1_dog` / `p2_dog` / `p3_dog`: whether or not the #x prediction is that breed of dog

In [702]:
# Check if there's any duplicated rows
print(sum(predictions.duplicated()))

# Check if there's any duplicated `tweet_id`
print(sum(predictions.duplicated('tweet_id')))

# Count number of duplicated `jpg_url` in predictions. (return True -> = 1, False = 0)
print(sum(predictions.duplicated('jpg_url')))

0
0
66


In [703]:
# Check if duplications are related to retweet?
sum(archive[archive['tweet_id'].isin(predictions[predictions.duplicated('jpg_url')]['tweet_id'])]['retweeted_status_id'].isna())
#  -> all duplicated records have values in retweets (not null) => the duplications are indeed retweets


0

In [704]:
# Check if the value in `px_conf` is <= 1?
sum(
    (predictions['p1_conf'] >1) |
    (predictions['p2_conf'] >1) |
    (predictions['p3_conf'] >1)
    )
# => No -> reasonable

0

### c. `tweet`

In [705]:
tweet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   created_at                     2354 non-null   datetime64[ns, UTC]
 1   id                             2354 non-null   int64              
 2   id_str                         2354 non-null   int64              
 3   full_text                      2354 non-null   object             
 4   truncated                      2354 non-null   bool               
 5   display_text_range             2354 non-null   object             
 6   entities                       2354 non-null   object             
 7   extended_entities              2073 non-null   object             
 8   source                         2354 non-null   object             
 9   in_reply_to_status_id          78 non-null     float64            
 10  in_reply_to_status_id_st

In [706]:
tweet.sample(5)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
1303,2016-03-09 02:08:59+00:00,707387676719185920,707387676719185920,Meet Clarkus. He's a Skinny Eastern Worcestershire. Can tie own shoes (impressive af) 10/10 would put on track team https://t.co/XP5o7zGn0E,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 707387669630881792, 'id_str': '707387669630881792', 'indices': [116, 139], 'media_url': 'http://pbs.twimg.com/media/CdElVm7XEAADP6o.jpg', 'media_url_https': 'https://pbs.twimg.com/media/CdElVm7XEAADP6o.jpg', 'url': 'https://t.co/XP5o7zGn0E', 'display_url': 'pic.twitter.com/XP5o7zGn0E', 'expanded_url': 'https://twitter.com/dog_rates/status/707387676719185920/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 576, 'h': 1024, 'resize': 'fit'}, 'small': {'w': 340, 'h': 604, 'resize': 'fit'}, 'medium': {'w': 576, 'h': 1024, 'resize': 'fit'}}}]}","{'media': [{'id': 707387669630881792, 'id_str': '707387669630881792', 'indices': [116, 139], 'media_url': 'http://pbs.twimg.com/media/CdElVm7XEAADP6o.jpg', 'media_url_https': 'https://pbs.twimg.com/media/CdElVm7XEAADP6o.jpg', 'url': 'https://t.co/XP5o7zGn0E', 'display_url': 'pic.twitter.com/XP5o7zGn0E', 'expanded_url': 'https://twitter.com/dog_rates/status/707387676719185920/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 576, 'h': 1024, 'resize': 'fit'}, 'small': {'w': 340, 'h': 604, 'resize': 'fit'}, 'medium': {'w': 576, 'h': 1024, 'resize': 'fit'}}}]}","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,NaN,NaN,None,"{'id': 4196983835, 'id_str': '4196983835', 'name': 'WeRateDogs™ (author)', 'screen_name': 'dog_rates', 'location': 'DM YOUR DOGS, WE WILL RATE', 'description': '#1 Source for Professional Dog Ratings | STORE: @ShopWeRateDogs | IG, FB & SC: WeRateDogs MOBILE APP: @GoodDogsGame | Business: dogratingtwitter@gmail.com', 'url': 'https://t.co/N7sNNHAEXS', 'entities': {'url': {'urls': [{'url': 'https://t.co/N7sNNHAEXS', 'expanded_url': 'http://weratedogs.com', 'display_url': 'weratedogs.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 3200947, 'friends_count': 104, 'listed_count': 2803, 'created_at': 'Sun Nov 15 21:41:29 +0000 2015', 'favourites_count': 114031, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 5288, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': '000000', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/861415328504569856/R2xOOfwe_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/861415328504569856/R2xOOfwe_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/4196983835/1501129017', 'profile_link_color': 'F5ABB5', 'profile_sidebar_border_color': '000000', 'profile_sidebar_fill_color': '000000', 'profile_text_color': '000000', 'profile_use_background_image': False, 'has_extended_profile': True, 'default_profile': False, 'default_profile_image': False, 'following': True, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none'}",NaN,NaN,None,NaN,False,1490,3835,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
849,2016-08-16 22:00:23+00:00

In [707]:
# Change name for better mapping later
new_column_names = {
    'id': 'tweet_id'
}

tweet.rename(columns=new_column_names, inplace = True)
# Minimize the dataset
tweet = tweet[['tweet_id', 'retweet_count', 'favorite_count', 'display_text_range']]

`tweet` columns:
- `tweet_id`: as mentioned
- `retweet_count`: number of times this tweet has been retweeted
- `favourite_count`: how many times this tweet has been liked by twitter users
- `display_text_range`: an array of 2 unicode code point, identifying the inclusive start and exclusive end of the displayable content of the tweet

### About the quality
`archive`:
- contains retweets. We only care about unique posts -> this might be considered duplication
- 281 records of tweet_id missing in `predictions`
- error datatypes: `in_reply_to_status_id`,`in_reply_to_user_id`,`timestamp`
- unnecessary html tags in `source` to differentiate utility name
-  `rating_numerator` has values <10 as well as some very large numbers
- `rating_denominator` has values other than 10
- wrong dog names starting with lowercase characters and glibberish (eg: a,an,actually,by)
- some records have more than one dog stages

`predictions`:
- After tidying, prediction number needs to have int type
- Value in `px` are inconsistent in the first letter capitalization
- Not all have dog-related prediction -> need to be dropped
- Duplicated `jpg_url` which are related to retweets
### About the tidiness
- `archive` without any duplicates (i.e: retweets) will have empty `retweeted_status_id`,`retweeted_status_user_id`,`retweeted_status_timestamp`, which can be dropped 
- `doggo, floofer, pupper and puppo` should be merged into one column named `stage`
- from 3 `px`, 1 should be picked then `breed` should be added in `archive`
- `retweet_count` and `favorite_count` from `tweet` should be joined with `archive`

# 3. Cleaning Data

### `archive` contains retweets. We only care about unique posts -> this might be considered duplication

**Define**

Keep only original tweet (i.e: retweeted_status_id is null). Delete the rest

**Code**

In [708]:
archive_clean =  archive.copy()
archive_clean = archive[archive['retweeted_status_id'].isnull()]

**Test**

In [709]:
# Check if there are any remaining not null records in retweeted_status_id
print(len(archive_clean[archive_clean['retweeted_status_id'].isnull() == False]))

# Check if there are any duplications of tweet_id 
sum(archive_clean.duplicated('tweet_id'))

#Pass test

0


0

### `prediction` Not all have dog-related prediction -> need to be dropped

**Define**

Only keep rows in which have at least 1 True value in `px_dog`

**Code**

In [710]:
predictions_clean = predictions.copy()

In [711]:
# Loop over px (x = 1,2,3) to check if at least 1 in 3 values is True
    # Create a mask value that have the same length with `predictions` and full of False
mask = pd.Series(False, index = predictions_clean.index)
    # Create an array of 1,2,3
num_columns = range(1,4,1)

for num in num_columns:
    column_name = f'p{num}_dog'
        # mask = False. If any values in the right operators are True -> mask = True -> also the line that have at least 1 True value
    mask |= predictions_clean[column_name] 

predictions_clean = predictions_clean[mask] # same as predictions_clean[mask == True]
predictions_clean

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2069,891087950875897856,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake_Bay_retriever,0.425595,True,Irish_terrier,0.116317,True,Indian_elephant,0.076902,False
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True


**Test**

In [712]:
# Check if there're any rows that not have at least 1 True
len(predictions_clean[
    (predictions_clean['p1_dog'] == False) &
    (predictions_clean['p2_dog'] == False) &
    (predictions_clean['p3_dog'] == False)
    ])

# Pass test

0

### 281 records of tweet_id missing in `predictions`

**Define**

Keep only records in `archive` that `tweet_id` exists in `predictions`

**Code**

In [713]:
archive_clean = archive_clean[archive_clean['tweet_id'].isin(predictions_clean['tweet_id'])]

**Test**

In [714]:
len(archive_clean[~archive_clean['tweet_id'].isin(predictions_clean['tweet_id'])])

0

### `archive` without any duplicates (i.e: retweets) will have empty `retweeted_status_id`,`retweeted_status_user_id`,`retweeted_status_timestamp`. Those columns can be dropped 

**Define**

Drop `retweeted_status_id`, `retweeted_status_user_id`,`retweeted_status_timestamp` from `archive`

**Code**

In [715]:
# Check if all values in those columns are empty
print(sum(~archive_clean['retweeted_status_id'].isna()))
print(sum(~archive_clean['retweeted_status_user_id'].isna()))
print(sum(~archive_clean['retweeted_status_timestamp'].isna()))

# Drop the columns
archive_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], axis=1, inplace = True)

0
0
0


**Test**

In [716]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1686 entries, 1 to 2355
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   tweet_id               1686 non-null   int64  
 1   in_reply_to_status_id  20 non-null     float64
 2   in_reply_to_user_id    20 non-null     float64
 3   timestamp              1686 non-null   object 
 4   source                 1686 non-null   object 
 5   text                   1686 non-null   object 
 6   expanded_urls          1686 non-null   object 
 7   rating_numerator       1686 non-null   int64  
 8   rating_denominator     1686 non-null   int64  
 9   name                   1267 non-null   object 
 10  doggo                  64 non-null     object 
 11  floofer                8 non-null      object 
 12  pupper                 176 non-null    object 
 13  puppo                  22 non-null     object 
 14  counter                1686 non-null   int64  
dtypes: float6

### error datatypes: `in_reply_to_status_id`,`in_reply_to_user_id`,`timestamp`

**Define**

Convert `in_reply_to_status_id`,`in_reply_to_user_id` to integer

Convert `timestamp` to datetime

**Code**

In [717]:
columns_to_int_type = ['in_reply_to_status_id','in_reply_to_user_id']

for column in columns_to_int_type:
    archive_clean[column] = archive_clean[column].fillna(0).astype(int)

archive_clean.timestamp = pd.to_datetime(archive_clean.timestamp)

**Test**

In [718]:
archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1686 entries, 1 to 2355
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id               1686 non-null   int64              
 1   in_reply_to_status_id  1686 non-null   int64              
 2   in_reply_to_user_id    1686 non-null   int64              
 3   timestamp              1686 non-null   datetime64[ns, UTC]
 4   source                 1686 non-null   object             
 5   text                   1686 non-null   object             
 6   expanded_urls          1686 non-null   object             
 7   rating_numerator       1686 non-null   int64              
 8   rating_denominator     1686 non-null   int64              
 9   name                   1267 non-null   object             
 10  doggo                  64 non-null     object             
 11  floofer                8 non-null      object             
 1

### `archive` unnecessary html tags in `source` to differentiate utility name

**Define**

Retain only the text between tags `<a...></a>`

Convert data type to categorical

**Code**

In [719]:
archive_clean.source.unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

In [720]:
# Extract the string in <a></a>
archive_clean.source = archive_clean.source.str.extract(r'<a[^>]*>(.*?)<\/a>', expand=False)

# Change type
archive_clean.source = archive_clean.source.astype('category')

**Test**

In [721]:
archive_clean['source'].value_counts()

source
Twitter for iPhone    1655
Twitter Web Client      22
TweetDeck                9
Name: count, dtype: int64

### `archive`: `rating_denominator` has values other than 10

**Define**

Check if the numerator and denominator have been correctly extracted from text or not. If not, re-extract and update

Check if after the update, the values of numerator and denominator are reasonable enough? (eg: denominator should be 10, and numerator should be >0 and not roughly higher than 10). If not, update

**Code**

In [722]:
# Count and check the alignment between fractions in `text` and the numerator/denominator
print(len(archive[archive['rating_denominator'] != 10]))
archive[archive['rating_denominator'] != 10][['text','rating_numerator', 'rating_denominator']].head()

# As shown in result, some fractions are not correctly extracted, therefore we need to update

23


,text,rating_numerator,rating_denominator
313,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",960,0
342,@docmisterio account started on 11/15/15,11,15
433,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
516,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24,7
784,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",9,11


In [723]:
# Skimming through the above table, many values in `text` has more than 1 fraction, and the current is taking the 1st one, which is often not the correct fraction.
# The correct fraction often the last one. So we will re-extract and update
# Define a function to extract the last fraction from a text
def extract_last_fraction(text):
    fraction_pattern = r'(\d+\.?\d*)/(\d+\.?\d*)'
    matches = re.findall(fraction_pattern,text) 
    if matches:
        last_match = matches[-1] #take the last fraction
        numerator = float(last_match[0])
        denominator = float(last_match[1])
        return numerator, denominator
    
# Apply the function to `text` and create new columns `correct_numerator` and `correct_denominator`
archive_clean['correct_numerator'], archive_clean['correct_denominator'] = zip(*archive_clean['text'].apply(extract_last_fraction))
  
archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,counter,correct_numerator,correct_denominator
1,892177421306343426,0,0,2017-08-01 00:17:27+00:00,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,NaN,NaN,NaN,NaN,0,13.0,10.0
2,891815181378084864,0,0,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,NaN,NaN,NaN,NaN,0,12.0,10.0
3,891689557279858688,0,0,2017-07-30 15:58:51+00:00,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,NaN,NaN,NaN,NaN,0,13.0,10.0
4,891327558926688256,0,0,2017-07-29 16:00:24+00:00,Twitter for iPhone,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,NaN,NaN,NaN,NaN,0,12.0,10.0
5,891087950875897856,0,0,2017-07-29 00:08:17+00:00,Twitter for iPhone,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,10,NaN,NaN,NaN,NaN,NaN,0,13.0,10.0


In [724]:
# Check if the correct denominator is more reasonable
print(archive_clean[archive_clean['rating_denominator'] == 10]['rating_denominator'].value_counts())
print(archive_clean[archive_clean['correct_denominator'] == 10]['correct_denominator'].value_counts())
len(archive_clean)

#So there are still x records not reasonable. 
# x=12
len(archive_clean)-len(archive_clean[archive_clean['correct_denominator'] == 10])

rating_denominator
10    1669
Name: count, dtype: int64
correct_denominator
10.0    1674
Name: count, dtype: int64


12

In [725]:
# Display the 12 not reasonable records:
with pd.option_context('display.max_colwidth', None):
    display(archive_clean[archive_clean['correct_denominator'] !=10][['text','correct_numerator','correct_denominator']])

,text,correct_numerator,correct_denominator
433,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84.0,70.0
516,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24.0,7.0
902,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165.0,150.0
1228,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99.0,90.0
1254,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.0,80.0
1274,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45.0,50.0
1351,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,60.0,50.0
1433,Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ,44.0,40.0
1634,"Two sneaky puppers were not initially seen, moving the rating to 143/130. Please forgive us. Thank you https://t.co/kRK51Y5ac3",143.0,130.0
1635,Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55,121.0,110.0


In [726]:
len(archive_clean['correct_denominator'] == 0)

1686

In [727]:
# Manually assessing, the fraction values in row 516 is not valid -> drop this row.
archive_clean.drop(index = 516, inplace=True)
# Also, all of the remaining fractions should be updated to the denominator equals to 10
archive_clean['rating_numerator'] = 10*archive_clean['correct_numerator']/archive_clean['correct_denominator']
archive_clean['rating_denominator'] = 10

**Test**

In [728]:
# Check if the fraction or numerator/denominator is equal to correct_numerator/correct_denominator
sum(archive_clean['rating_numerator']/archive_clean['rating_denominator'] != (archive_clean['correct_numerator']/archive_clean['correct_denominator']))

# -> all are equal

#Drop the `correct_numerator` and `correct_denominator` columns
archive_clean.drop(columns=['correct_numerator','correct_denominator'], inplace=True)

### `archive`: wrong dog names starting with lowercase characters and glibberish (eg: a,an,actually,by)

**Define**

Replace all lowercase values in `name` with None


**Code**

In [729]:
# Only keep records starts with uppercase and followed by at least 1 lowercase. Other mark as NaN
archive_clean['name'] = archive_clean['name'].apply(lambda row: row if pd.notna(row) and re.match(r'[A-Z][a-z]+', row) else np.nan)

**Test**

In [730]:
print(archive_clean['name'].unique())

archive_clean['name'].to_string().islower()

# All the names seem to be correct

['Tilly' 'Archie' 'Darla' 'Franklin' nan 'Jax' 'Zoey' 'Cassie' 'Koda'
 'Bruno' 'Ted' 'Stuart' 'Oliver' 'Jim' 'Zeke' 'Ralphus' 'Gerald' 'Jeffrey'
 'Canela' 'Maya' 'Mingus' 'Roscoe' 'Waffles' 'Jimbo' 'Maisey' 'Earl'
 'Lola' 'Kevin' 'Yogi' 'Noah' 'Bella' 'Grizzwald' 'Rusty' 'Gus' 'Stanley'
 'Alfy' 'Koko' 'Rey' 'Gary' 'Jesse' 'Romeo' 'Bailey' 'Duddles' 'Jack'
 'Beau' 'Snoopy' 'Shadow' 'Emmy' 'Aja' 'Penny' 'Dante' 'Nelly' 'Ginger'
 'Benedict' 'Venti' 'Goose' 'Nugget' 'Cash' 'Jed' 'Sebastian' 'Sierra'
 'Monkey' 'Harry' 'Kody' 'Lassie' 'Rover' 'Napolean' 'Boomer' 'Cody'
 'Rumble' 'Dewey' 'Scout' 'Gizmo' 'Walter' 'Cooper' 'Harold' 'Shikha'
 'Lili' 'Jamesy' 'Coco' 'Sammy' 'Meatball' 'Paisley' 'Neptune' 'Belle'
 'Quinn' 'Zooey' 'Dave' 'Hobbes' 'Burt' 'Lorenzo' 'Carl' 'Milky' 'Trooper'
 'Sophie' 'Wyatt' 'Rosie' 'Thor' 'Oscar' 'Callie' 'Cermet' 'Marlee' 'Arya'
 'Einstein' 'Alice' 'Benny' 'Aspen' 'Jarod' 'Sailor' 'Snoop' 'Kyle' 'Leo'
 'Riley' 'Noosh' 'Odin' 'Jerry' 'Rontu' 'Cannon' 'Furzey' 'Daisy'

False

### `archive`: some records have more than one dog stage

**Define**

Check out the reason why and correct to only 1 stage. If can't, update the stages column

Create a `stage` column to migrate all 4 stage columns into 1.

In [731]:
# From the above, we already separated number of stage in each row with the column `counter`.
# Display only records with multiple stages
archive_clean[archive_clean['counter'] == 2][['text','counter']]

# Will deal with this later after creating `stage`

,text,counter
191,Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel,2
200,"At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk",2
460,"This is Dido. She's playing the lead role in ""Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple."" 13/10 (IG: didodoggo) https://t.co/m7isZrOBX7",2
531,Here we have Burke (pupper) and Dexter (doggo). Pupper wants to be exactly like doggo. Both 12/10 would pet at same time https://t.co/ANBpEYHaho,2
565,"Like doggo, like pupper version 2. Both 11/10 https://t.co/9IxWAXFqze",2
575,This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths pupper everything's fine https://t.co/55Dqe0SJNj,2
889,"Meet Maggie &amp; Lila. Maggie is the doggo, Lila is the pupper. They are sisters. Both 12/10 would pet at the same time https://t.co/MYwR4DQKll",2
956,Please stop sending it pictures that don't even have a doggo or pupper in them. Churlish af. 5/10 neat couch tho https://t.co/u2c9c7qSg8,2
1063,This is just downright precious af. 12/10 for both pupper and doggo https://t.co/o5J479bZUC,2
1113,"Like father (doggo), like son (pupper). Both 12/10 https://t.co/pG2inLaOda",2


In [732]:
# Check if there's any glibberish values in 4 columns
print(
    archive_clean['doggo'].unique(),
    archive_clean['floofer'].unique(),
    archive_clean['pupper'].unique(),
    archive_clean['puppo'].unique()
    )
# -> No

# Create a `stage` column
dog_stage = pd.melt(archive_clean,
                    id_vars='tweet_id',
                    value_vars=['doggo','floofer','pupper','puppo'],
                    var_name='stage_name',
                    value_name='stage')


[nan 'doggo'] [nan 'floofer'] [nan 'pupper'] [nan 'puppo']


In [733]:
dog_stage.valuest.unique()

AttributeError: 'DataFrame' object has no attribute 'value'

In [ ]:
dog_stage

,tweet_id,stage,value
0,892177421306343426,doggo,NaN
1,891815181378084864,doggo,NaN
2,891689557279858688,doggo,NaN
3,891327558926688256,doggo,NaN
4,891087950875897856,doggo,NaN
...,...,...,...
6735,666049248165822465,puppo,NaN
6736,666044226329800704,puppo,NaN
6737,666033412701032449,puppo,NaN
6738,666029285002620928,puppo,NaN
